# Modelo no. 2
## CNN
### Predicción de argumentos efectivos
    
Grupo:
- Cristian Aguirre: 20231
- Diego Córdova: 20212
- Marco Jurado: 20308
- Paola Contreras: 20213
- Paola de León: 20361

In [50]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Carga de Datos de Entreno

In [51]:
id = '1kzPayZj888s0RkHlxYHGXzHwdb63fEYH'
url = 'https://drive.google.com/uc?id=' + id
data = pd.read_csv(url)
data.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate


## Liempieza de dataset de entreno

Se eliminaran las columnas de ***discourse_id*** y ***essay_id*** debido a que estas son solo identificadores no relevantes para el modelo. AL contrario, podrian causar ruido que genere malas predicciones.

Ademas se agrega una columna ***index*** y se guarda la columna ***discourse_text*** en una variable aparte ya que esta servira como entrada de la capa de word embedding.

In [52]:
data.drop('discourse_id', axis=1, inplace=True)
data.drop('essay_id', axis=1, inplace=True)
data['index'] = data.index
data.head()

,discourse_text,discourse_type,discourse_effectiveness,index
0,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,0
1,"On my perspective, I think that the face is a ...",Position,Adequate,1
2,I think that the face is a natural landform be...,Claim,Adequate,2
3,"If life was on Mars, we would know by now. The...",Evidence,Adequate,3
4,People thought that the face was formed by ali...,Counterclaim,Adequate,4


In [53]:
claim_sizes = [len(text) for text in data['discourse_text']]
data['claim_size'] = claim_sizes
data.head()

,discourse_text,discourse_type,discourse_effectiveness,index,claim_size
0,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,0,317
1,"On my perspective, I think that the face is a ...",Position,Adequate,1,210
2,I think that the face is a natural landform be...,Claim,Adequate,2,105
3,"If life was on Mars, we would know by now. The...",Evidence,Adequate,3,362
4,People thought that the face was formed by ali...,Counterclaim,Adequate,4,101


In [54]:
texto_original = data['discourse_text']
data.drop('discourse_text', axis=1, inplace=True)
data.head()

,discourse_type,discourse_effectiveness,index,claim_size
0,Lead,Adequate,0,317
1,Position,Adequate,1,210
2,Claim,Adequate,2,105
3,Evidence,Adequate,3,362
4,Counterclaim,Adequate,4,101


### Encoding de variables categoricas
En este caso las variables ***discourse_effectiveness*** y ***discourse_type*** son categoricas.

In [55]:
type_map = {cat:index for index, cat in enumerate(data['discourse_type'].unique())}
print('> mapa para encoding de discourse_type', type_map)
data['discourse_type'] = [type_map[cat] for cat in data['discourse_type']]
data[['discourse_type']].head()

> mapa para encoding de discourse_type {'Lead': 0, 'Position': 1, 'Claim': 2, 'Evidence': 3, 'Counterclaim': 4, 'Rebuttal': 5, 'Concluding Statement': 6}


,discourse_type
0,0
1,1
2,2
3,3
4,4


In [56]:
type_map = {cat:index for index, cat in enumerate(data['discourse_effectiveness'].unique())}
print('> mapa para encoding de discourse_effectiveness', type_map)
data['discourse_effectiveness'] = [type_map[cat] for cat in data['discourse_effectiveness']]
data[['discourse_effectiveness']].head()

> mapa para encoding de discourse_effectiveness {'Adequate': 0, 'Ineffective': 1, 'Effective': 2}


,discourse_effectiveness
0,0
1,0
2,0
3,0
4,0


In [57]:
data.head()

,discourse_type,discourse_effectiveness,index,claim_size
0,0,0,0,317
1,1,0,1,210
2,2,0,2,105
3,3,0,3,362
4,4,0,4,101


## Generacion de Secuencias de Texto

In [58]:
texto_original

0        Hi, i'm Isaac, i'm going to be writing about h...
1        On my perspective, I think that the face is a ...
2        I think that the face is a natural landform be...
3        If life was on Mars, we would know by now. The...
4        People thought that the face was formed by ali...
                               ...                        
36760    For many people they don't like only asking on...
36761    also people have different views and opinions ...
36762    Advice is something that can impact a persons ...
36763    someone can use everything that many people sa...
36764    In conclusion asking for an opinion can be ben...
Name: discourse_text, Length: 36765, dtype: object

In [59]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from copy import deepcopy as dcp

# Tokenizar las frases
tokenizador = Tokenizer()
tokenizador.fit_on_texts(texto_original)
secuencias = tokenizador.texts_to_sequences(texto_original)
    
# Rellenar (Pad) las secuencias para que tengan la misma longitud
secuencias = pad_sequences(secuencias)

# Dimensiones de entrada y salida de la capa de inscrustamiento

long_vocab = len(tokenizador.word_index) + 1
dim_incrustamiento = 2  # Representar cada palabra por un vector 2D

pd.DataFrame(secuencias).head()

,0,1,2,3,4,5,6,7,8,9,...,836,837,838,839,840,841,842,843,844,845
0,0,0,0,0,0,0,0,0,0,0,...,17,155,25,24,9,8,57,3,343,300
1,0,0,0,0,0,0,0,0,0,0,...,59,69,18,58,6,8,8,3,343,300
2,0,0,0,0,0,0,0,0,0,0,...,8,86,120,17,155,6,30,14,8571,536
3,0,0,0,0,0,0,0,0,0,0,...,7,39,6,508,436,105,381,3,343,300
4,0,0,0,0,0,0,0,0,0,0,...,7603,28,10,398,6,38,50,120,17,155


## Split de Datos

In [60]:
y = data['discourse_effectiveness']
X = data.copy()
X.drop('discourse_effectiveness', axis=1, inplace=True)

In [61]:
from sklearn.model_selection import train_test_split

# Dividir los datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,discourse_type,index,claim_size
3994,3,3994,472
3069,6,3069,151
4749,2,4749,67
34914,3,34914,238
2878,2,2878,19
...,...,...,...
16850,3,16850,100
6265,2,6265,47
11284,0,11284,223
860,3,860,209


Se usara la columna ***index*** que creamos anteriormente para dividir los datos de texto tokenizados

In [62]:
secuencias_train = [sec for index, sec in enumerate(secuencias) if index in X_train['index']]
secuencias_test = [sec for index, sec in enumerate(secuencias) if index in X_test['index']]

Ahora se borra la columna ***index*** en los datasdets "X" luego de usarla para hacer el split en las secuencias

In [63]:
X_train.drop('index', axis=1, inplace=True)
X_test.drop('index', axis=1, inplace=True)
X_train

,discourse_type,claim_size
3994,3,472
3069,6,151
4749,2,67
34914,3,238
2878,2,19
...,...,...
16850,3,100
6265,2,47
11284,0,223
860,3,209


Se pasan a arrays de numpy para darle como input al modelo

In [64]:
X_train = X_train.values
X_test = X_test.values
X_train

array([[  3, 472],
       [  6, 151],
       [  2,  67],
       ...,
       [  0, 223],
       [  3, 209],
       [  6, 339]], dtype=int64)

Por ultimo, se convierten las secuencias a tensores para agregarlas como input al modelo y las variables objetivo ***y_test*** y ***y_train*** se convierten a arrays de numpy

In [65]:
import tensorflow as tf

secuencias_train_list = tf.stack(secuencias_train)
secuencias_test_list = tf.stack(secuencias_test)
y_train = y_train.values
y_test = y_test.values
secuencias_train_list

<tf.Tensor: shape=(29412, 846), dtype=int32, numpy=
array([[   0,    0,    0, ...,    3,  343,  300],
       [   0,    0,    0, ...,    3,  343,  300],
       [   0,    0,    0, ...,   14, 8571,  536],
       ...,
       [   0,    0,    0, ...,   41,   13,  130],
       [   0,    0,    0, ...,   73,  115,  289],
       [   0,    0,    0, ...,    2,  163,  138]])>

### Modelo con CNN

In [66]:

# Construcción del modelo CNN
model = Sequential()

model.add(Embedding(
    input_dim=long_vocab,
    output_dim=4,
    input_length=len(secuencias),
))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Conv1D(256, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Capa de salida para clasificación binaria

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 846, 128)          108288    
                                                                 
 conv1d_9 (Conv1D)           (None, 842, 64)           41024     
                                                                 
 max_pooling1d_6 (MaxPoolin  (None, 421, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_10 (Conv1D)          (None, 417, 128)          41088     
                                                                 
 max_pooling1d_7 (MaxPoolin  (None, 208, 128)          0         
 g1D)                                                            
                                                                 
 conv1d_11 (Conv1D)          (None, 204, 256)         

In [67]:
print('Largo secuencias de entreno', len(secuencias_train))
print('Largo matriz de entreno:', len(X_train))
assert len(secuencias_train) == len(X_train)

Largo secuencias de entreno 29412
Largo matriz de entreno: 29412


In [69]:
model.fit(secuencias_train_list, y_train, epochs=10, batch_size=32, validation_data=(secuencias_test_list, y_test))

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_3/embedding_3/embedding_lookup' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "C:\Users\cor20212\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\cor20212\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1053, in launch_instance
      app.start()
    File "C:\Users\cor20212\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\kernelapp.py", line 737, in start
      self.io_loop.start()
    File "C:\Users\cor20212\AppData\Local\Packages\PythonSoftw